# Training a model for catchment classification. 

# Imports

In [73]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

desired_width = 500
pd.set_option("display.width", desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option("display.max_columns", 30)

In [4]:
classes = pd.DataFrame(
    data={
        "classes": [
            "compact_urban_development",
            "urban",
            "loose_urban_development",
            "wooded_area",
            "grassy",
            "loose_soil",
            "steep_area",
        ]
    }
)
classes

,classes
0,compact_urban_development
1,urban
2,loose_urban_development
3,wooded_area
4,grassy
5,loose_soil
6,steep_area


# Get files

In [5]:
INP_FILE = "dataset/subcatchment_dataset.inp"
RPT_FILE = "dataset/subcatchment_dataset.inp"

# Run simulation

In [15]:
with pyswmm.Simulation(FILE) as sim:
    for step in sim:
        pass

# Read inp and rpt file as swmmio model object

In [6]:
model = swmmio.Model(INP_FILE)

## Get subcatchments data from the model

In [7]:
raw_subcatchments = model.subcatchments.dataframe
subcatchments = raw_subcatchments.copy()
subcatchments[:5]

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,RouteTo,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,coords
Name,,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,OUTLET,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,"[(777180.0, 592590.0), (777180.0, 592585.0), (777175.0, 592585.0), (777175.0, 592590.0), (777180.0, 592590.0)]"
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,NaN
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,NaN
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,OUTLET,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224,NaN
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,OUTLET,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879,NaN


### Drop unused columns

In [8]:
subcatchments.drop(['coords', 'RouteTo'], axis=1, inplace=True)
subcatchments

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S995,Raingage2,O1,1.10,10.00,104.88,15.00,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.53,0.90,1.56,2.46,0.03,0.02,0.243
S996,Raingage2,O1,1.89,5.09,137.48,15.00,0,0.060,0.17,1.27,5.08,10,10.10,0.00,0.00,7.55,0.46,2.04,2.49,0.05,0.03,0.247
S997,Raingage2,O1,1.19,10.00,109.09,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.91,0.90,1.18,2.08,0.02,0.01,0.206


## Get categories

In [9]:
categories = model.inp.tags
categories

,Name,Tag
ElementType,,
Subcatch,S1,#comment
Subcatch,S2,compact_urban_development
Subcatch,S3,loose_soil
Subcatch,S4,wooded_area
Subcatch,S5,loose_urban_development
...,...,...
Subcatch,S997,grassy
Subcatch,S998,steep_area
Subcatch,S999,urban


### Add categories column to subcatchments DataFrame

In [10]:
# Reset the index of subcatchments DataFrame
subcatchments.reset_index(inplace=True)

# Merge the two DataFrames on the "Name" column
merged_df = subcatchments.merge(model.inp.tags, left_on="Name", right_on="Name", how="left")

# Rename the 'Tag' column to 'categories'
merged_df.rename(columns={"Tag": "categories"}, inplace=True)

# Set the index back to "Name"
merged_df.set_index("Name", inplace=True)

In [11]:
merged_df[:3]

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
Name,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.0,300.00,5.0,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,#comment
S10,Raingage2,O1,1.87,45.0,136.75,15.0,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,compact_urban_development
S100,Raingage2,O1,1.71,45.0,130.77,10.0,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,compact_urban_development


In [12]:
merged_df.head()

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
Name,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,#comment
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,compact_urban_development
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,compact_urban_development
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224,grassy
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879,steep_area


### Split data into features and target

In [13]:
X = merged_df.drop('categories', axis=1)
y = merged_df['categories']
print(X.shape, y.shape)

(1001, 22) (1001,)


In [14]:
X["TotalPrecip"] = pd.to_numeric(X["TotalPrecip"])
X["TotalRunon"] = pd.to_numeric(X["TotalRunon"])
X["TotalEvap"] = pd.to_numeric(X["TotalEvap"])
X["TotalInfil"] = pd.to_numeric(X["TotalInfil"])
X["PervRunoff"] = pd.to_numeric(X["PervRunoff"])

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, S1 to S999
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Raingage       1001 non-null   object 
 1   Outlet         1001 non-null   object 
 2   Area           1001 non-null   float64
 3   PercImperv     1001 non-null   float64
 4   Width          1001 non-null   float64
 5   PercSlope      1001 non-null   float64
 6   CurbLength     1001 non-null   int64  
 7   N-Imperv       1001 non-null   float64
 8   N-Perv         1001 non-null   float64
 9   S-Imperv       1001 non-null   float64
 10  S-Perv         1001 non-null   float64
 11  PctZero        1001 non-null   int64  
 12  TotalPrecip    1001 non-null   float64
 13  TotalRunon     1001 non-null   float64
 14  TotalEvap      1001 non-null   float64
 15  TotalInfil     1001 non-null   float64
 16  ImpervRunoff   1001 non-null   float64
 17  PervRunoff     1001 non-null   float64
 18  TotalRunoffI

In [16]:
X = pd.get_dummies(X, drop_first=True)
X.head()

,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,
S1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,10.1,0.0,0.0,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,10.1,0.0,0.0,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,10.1,0.0,0.0,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,10.1,0.0,0.0,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,10.1,0.0,0.0,1.04,8.23,0.65,8.88,0.02,0.02,0.879


In [17]:
# Create a MinMaxScaler instance
scaler = MinMaxScaler()

# Fit the scaler to the data and transform the data
X_normalized = scaler.fit_transform(X)

# Convert the normalized data back to a dataframe with the same column names
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns, index=X.index)

In [18]:
X_normalized_df.head()

,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,,,,,,,,,,
S1,0.246231,0.218148,1.000000,0.044995,0.0,0.354005,0.480000,0.0,0.0,0.764706,0.0,0.0,0.0,0.608479,0.216687,0.683871,0.392630,0.111111,0.142857,0.391528
S10,0.934673,0.526790,0.437069,0.213515,0.0,0.000000,0.133333,0.0,0.0,1.000000,0.0,0.0,0.0,0.386534,0.534247,0.445161,0.622618,0.611111,0.642857,0.622593
S100,0.854271,0.526790,0.416448,0.129255,0.0,0.000000,0.133333,0.0,0.0,1.000000,0.0,0.0,0.0,0.392768,0.534247,0.429032,0.614994,0.555556,0.571429,0.614891
S1000,0.336683,0.094691,0.249862,0.046512,0.0,0.354005,0.480000,0.0,0.0,0.058824,0.0,0.0,0.0,0.835411,0.085928,0.422581,0.156290,0.111111,0.071429,0.156611
S1001,0.125628,1.000000,0.141345,0.719077,0.0,0.000000,0.000000,0.0,0.0,0.882353,0.0,0.0,0.0,0.001247,0.998755,0.193548,0.997459,0.111111,0.142857,0.997433


### Encode target labels

In [23]:
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

In [24]:
y_encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

# Split the data into training and testing sets

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized_df, y_encoded, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (800, 20)
y_train shape: (800, 8)
X_test shape: (201, 20)
y_test shape: (201, 8)


# Build the model

In [71]:
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 16)                336       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_31 (Dense)            (None, 8)                 136       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_32 (Dense)            (None, 8)                 72        
                                                                 
Total params: 544
Trainable params: 544
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2,
    verbose=1,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)

Epoch 1/100
50/50 [==============================] - 1s 5ms/step - loss: 0.3344 - accuracy: 0.8650 - val_loss: 0.0827 - val_accuracy: 1.0000
Epoch 2/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2901 - accuracy: 0.8875 - val_loss: 0.0813 - val_accuracy: 1.0000
Epoch 3/100
50/50 [==============================] - 0s 2ms/step - loss: 0.3190 - accuracy: 0.8712 - val_loss: 0.0781 - val_accuracy: 1.0000
Epoch 4/100
50/50 [==============================] - 0s 2ms/step - loss: 0.3258 - accuracy: 0.8687 - val_loss: 0.0832 - val_accuracy: 1.0000
Epoch 5/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2931 - accuracy: 0.8838 - val_loss: 0.0810 - val_accuracy: 1.0000
Epoch 6/100
50/50 [==============================] - 0s 2ms/step - loss: 0.3280 - accuracy: 0.8600 - val_loss: 0.0808 - val_accuracy: 1.0000
Epoch 7/100
50/50 [==============================] - 0s 2ms/step - loss: 0.3228 - accuracy: 0.8737 - val_loss: 0.0811 - val_accuracy: 1.0000
Epoch 8/100
5

In [67]:
# Evaluate ANN model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {accuracy:.2f}")

7/7 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Test set accuracy: 1.00


In [ ]:
# model.predict(X_test)
y_pred = model.predict(X_test)